In [19]:
import scipy.io as sio
import numpy as np
from sklearn.decomposition import NMF
import math
import tensorflow as tf
import graph
import utils_adv
import scipy.io as sio

import multiprocessing
from sklearn.decomposition import NMF
import os

name = "cora"
name_dataset = './data'
#num_sample_per_walk = 10
#dim = 150
num_poly = 7
#window_size = 4
#path_length = 11
num_neg = 15
#num_walks_per_node = 2

dim = 20
window_size = 5
num_sample_per_walk = 5
num_walks_per_node = 10
path_length = 80

data_set = name+".mat"
input_net = name_dataset+"/"+ data_set

folder_dataset = os.path.join(os.path.dirname(os.getcwd()), 'data', data_set)
appendix_this_run = str(dim) + '_' + str(num_poly) + '_' + str(window_size) + '_' + str(num_neg) + '_' + str(num_walks_per_node)# + '_' + str(num_sample_per_walk)
appendix_path_this_run = str(window_size) + '_' + str(num_walks_per_node)
fn_result_emb = name_dataset+'/embd_poly_'+appendix_this_run+'.embeddings'


network = sio.loadmat(input_net)['network']
num_nodes = network.shape[0]

network_2 = np.dot(network,network)
network_3 = np.dot(network_2,network)
network_4 = np.dot(network_3,network)
network_5 = np.dot(network_4,network)
network_6 = np.dot(network_5,network)
network_7 = np.dot(network_6,network)
order_network = (network + network_2 + network_3 + network_4 + network_5 + network_6+ network_7) / 7 
model = NMF(n_components=num_poly, init='random', random_state=0)
W = model.fit_transform(order_network)
H = model.components_

fn_clustering = W




dim = 7
model = NMF(n_components=dim, init='random', random_state=0)
W = model.fit_transform(order_network)
H = model.components_
vectors = W


norw_W = np.zeros(W.shape)
fill_norm = np.ones((1,dim)) / dim
for i in range(W.shape[0]):
    if np.sum(W[i,:]) == 0:
        norw_W[i] = fill_norm
    else:
        norw_W[i] = W[i,:] / np.sum(W[i,:])
vectors = norw_W



input_net = 'data/cora.mat'
directed = 'directed'
weighted = 'weighted'
p = 1
q = 1
window_size = 5
negative = 1
num_walks = 50
walk_length = 30
network = sio.loadmat(input_net)['network']
G_adv = graph.Graph(input_net, directed, weighted, p, q, flag=True)
G_randomwalk = G_adv.simulate_walks(num_walks, walk_length)
#edges, neg_pairs = utils_adv.walks_to_pairs_with_ns_pool(G_randomwalk, window_size, negative, G_adv.table)

dict_keys(['__header__', '__version__', '__globals__', 'network', 'group'])
Filling unigram table
Finish filling unigram table
Walk iteration:
1/50
2/50
3/50
4/50
5/50
6/50
7/50
8/50
9/50
10/50
11/50
12/50
13/50
14/50
15/50
16/50
17/50
18/50
19/50
20/50
21/50
22/50
23/50
24/50
25/50
26/50
27/50
28/50
29/50
30/50
31/50
32/50
33/50
34/50
35/50
36/50
37/50
38/50
39/50
40/50
41/50
42/50
43/50
44/50
45/50
46/50
47/50
48/50
49/50
50/50


In [20]:
topK_node = {}
node_num = W.shape[0]
for i in range(node_num):
    tmp = list(norw_W[i])
    topK_node[i] = tmp.index(max(tmp))

In [21]:
global _G_randomwalk
global _norw_W
global _index
_G_randomwalk = G_randomwalk
_norw_W = norw_W

def _sampe_topic(n):
    walk = _G_randomwalk[n, :]
    return [int(np.random.choice(_index, 1, p = _norw_W[step])) for step in walk]


from time import time
from multiprocessing import Pool
from multiprocessing import cpu_count

#sentence_topics = []
index = range(dim)

_index = index
pool = Pool(cpu_count())
sentence_topics = pool.map(_sampe_topic, range(_G_randomwalk.shape[0]))
pool.close()
pool.join()

# for walk in G_randomwalk:
#     sentence_topics.append([int(np.random.choice(index, 1, p = norw_W[step])) for step in walk])

sentence_topics = np.array(sentence_topics)

In [22]:
def cum_walks_to_pairs_with_ns_pool(norw_W, walks, window_size, negative, table, sentence_topics, topic_dim):
    
    begin_time = time()
    global _window_size
    global _negative
    global _table
    global _walks
    global _walk_length
    global _sentence_topics
    global _topic_dim
    global _norw_W
    global _index
    
    _sentence_topics = sentence_topics
    _window_size = window_size
    _negative = negative
    _table = table
    _walks = walks
    _walk_length = walks.shape[1]
    _topic_dim = topic_dim
    _norw_W = norw_W
    _index = range(topic_dim)

    pool = Pool(cpu_count())
    pairs = pool.map(_cum_walk_to_pairs, range(walks.shape[0]))
    pool.close()
    pool.join()

    pos_pairs, neg_pairs = [], []
    for i in range(len(pairs)):
        pos_pairs.append(pairs[i][0])
        neg_pairs.append(pairs[i][1])

    print("walks_to_pairs_with_ns_pool [%.1f s]" % (time() - begin_time))
    return np.concatenate(pos_pairs, axis=0), np.concatenate(neg_pairs, axis=0)

def _cum_walk_to_pairs(n):
    pos_pairs, neg_pairs = [], []

    walk = _walks[n, :]
    
    target_topic = np.random.choice(_sentence_topics[n])
    for l in range(_walk_length):
        for m in range(l-_window_size, l+_window_size+1):
            if m<0 or m>=_walk_length: continue
            
            neg_topic = np.random.choice(_topic_dim)
            #neg_topic = np.random.choice(_index, p = _norw_W[walk[l]])
            while target_topic == neg_topic:
                neg_topic = np.random.choice(_topic_dim)
                #neg_topic = np.random.choice(_index, p = _norw_W[walk[l]])
            
            pos_pairs.append([walk[l], walk[m], target_topic, neg_topic])
            for k in range(_negative):
                n_neg = _table.sample(1)[0]
                while n_neg==walk[l] or n_neg==walk[m]:
                    n_neg = _table.sample(1)[0]
                neg_pairs.append([walk[l], n_neg])

    return np.array(pos_pairs, dtype=np.int32), np.array(neg_pairs, dtype=np.int32)

In [23]:
from time import time
from multiprocessing import Pool
from multiprocessing import cpu_count

topic_dim = W.shape[1]
edges, neg_pairs = cum_walks_to_pairs_with_ns_pool(norw_W, G_randomwalk, window_size, negative, G_adv.table, sentence_topics, topic_dim)

walks_to_pairs_with_ns_pool [54.6 s]


In [24]:
sen_i = 21
print(np.random.choice(sentence_topics[sen_i]))
print(sentence_topics[sen_i])

5
[1 5 6 5 3 5 5 5 4 5 5 5 5 5 1 5 1 1 1 1 1 1 5 1 1 1 2 5 1 5]


In [25]:
class Discriminator(object):
    def __init__(self, n_node, node_emd_init, topic_emb_init, lr_dis, id):
        
        self.n_node = n_node
        self.node_emb_init = tf.constant(node_emb_init, tf.float32)
        self.topic_emb_init = tf.constant(topic_emb_init, tf.float32)

        with tf.variable_scope('Dis_Embeddings_{}'.format(id)):

            self.u_node_embed  = tf.get_variable(name='u_node_embed', shape= node_emb_init.shape, 
                                              initializer=tf.constant_initializer(node_emb_init), trainable=True)
            
            self.v_node_embed  = tf.get_variable(name='v_node_embed', shape= node_emb_init.shape, 
                                              initializer=tf.constant_initializer(node_emb_init), trainable=True)
            
            self.topic_embed  = tf.get_variable(name='node_topic_embed', shape= topic_emb_init.shape, 
                                              initializer=tf.constant_initializer(topic_emb_init), trainable=True)
            
            
            #self.bias_vector = tf.Variable(tf.ones([self.n_node]))
            
        with tf.name_scope('Dis_Input'):
            self.target_node = tf.placeholder(tf.int32, shape=[None], name='target')
            self.pos_node = tf.placeholder(tf.int32, shape=[None], name='positive') 
            self.neg_node = tf.placeholder(tf.int32, shape=[None], name= 'negative')
            self.target_topic = tf.placeholder(tf.int32, shape=[None], name= 'target_topic')
            self.neg_topic = tf.placeholder(tf.int32, shape=[None], name= 'neg_topic')
            
        with tf.name_scope('Embedding_Lookup'):
            self.target_emb = tf.nn.embedding_lookup(self.u_node_embed, self.target_node)
            self.pos_emb = tf.nn.embedding_lookup(self.v_node_embed, self.pos_node)
            self.neg_emb = tf.nn.embedding_lookup(self.v_node_embed, self.neg_node)
            self.target_topic_emb = tf.nn.embedding_lookup(self.topic_embed, self.target_topic)
            #self.bias = tf.gather(self.bias_vector, self.target_node)
            self.neg_topic_emb = tf.nn.embedding_lookup(self.topic_embed, self.neg_topic)
            
        with tf.name_scope('Dis_Reward'):
            neg_topic_score = tf.reduce_sum(-1.0 * self.target_emb * self.neg_topic_emb, axis=1)
            #neg_topic_score = tf.reduce_sum(-1.0 * self.pos_emb * self.neg_topic_emb, axis=1)
            
            self.reward = -tf.log(tf.clip_by_value(tf.nn.sigmoid(neg_topic_score), 1e-5, 1))
            
        with tf.name_scope('Dis_Loss'):
            #self.pos_score = tf.reduce_sum(self.target_emb * self.pos_emb, axis=1) + self.bias
            self.pos_score = tf.reduce_sum(self.target_emb * self.pos_emb, axis=1)
            self.pos_prob = tf.clip_by_value(tf.nn.sigmoid(self.pos_score), 1e-5, 1)
            
            #self.neg_score = tf.reduce_sum(-1.0 * self.target_emb * self.neg_emb, axis=1) + self.bias
            self.neg_score = tf.reduce_sum(-1.0 * self.target_emb * self.neg_emb, axis=1) 
            self.neg_prob = tf.clip_by_value(tf.nn.sigmoid(self.neg_score), 1e-5, 1)
            
            #self.target_topic_score = tf.reduce_sum(self.pos_emb * self.target_topic_emb, axis=1) + self.bias
            #self.target_topic_score = tf.reduce_sum(self.pos_emb * self.target_topic_emb, axis=1)
            
            #self.target_topic_score = tf.reduce_sum(self.target_emb * self.target_topic_emb, axis=1)
            self.target_topic_score = tf.reduce_sum(self.pos_emb * self.target_topic_emb, axis=1)
            self.target_topic_prob = tf.clip_by_value(tf.nn.sigmoid(self.target_topic_score), 1e-5, 1)
            
            #self.neg_topic_score = tf.reduce_sum(-1.0 * self.neg_topic_emb * self.target_topic_emb, axis=1)
            self.neg_topic_score = tf.reduce_sum(-1.0 * self.target_emb * self.neg_topic_emb, axis=1)
            #self.neg_topic_score = tf.reduce_sum(-1.0 * self.pos_emb * self.neg_topic_emb, axis=1)
            self.neg_topic_prob = tf.clip_by_value(tf.nn.sigmoid(self.neg_topic_score), 1e-5, 1)
            
            self.dis_loss = -tf.reduce_mean(tf.log(self.neg_topic_prob))-tf.reduce_mean(tf.log(self.pos_prob))-tf.reduce_mean(tf.log(self.neg_prob))-tf.reduce_mean(tf.log(self.target_topic_prob)) + 1e-5 * (tf.nn.l2_loss(self.neg_topic_emb)+tf.nn.l2_loss(self.target_emb)+tf.nn.l2_loss(self.pos_emb) + tf.nn.l2_loss(self.neg_emb)+tf.nn.l2_loss(self.target_topic_emb))

            #self.dis_loss = -tf.reduce_mean(tf.log(self.pos_prob)) -tf.reduce_mean(tf.log(self.neg_prob))+ 1e-5 * (tf.nn.l2_loss(self.target_emb)+tf.nn.l2_loss(self.pos_emb) + tf.nn.l2_loss(self.neg_emb))
            
        with tf.name_scope('Dis_Train'):
            optimizer = tf.train.AdamOptimizer(lr_dis)
            self.dis_update = optimizer.minimize(self.dis_loss)

In [26]:
class Generator(object):
    def __init__(self, n_node, node_emd_init, topic_emb_init, lr_gen, id):
        
        self.n_node = n_node
        self.node_emb_init = tf.constant(node_emb_init, tf.float32)
        self.topic_emb_init = tf.constant(topic_emb_init, tf.float32)
        self.topic_num = topic_emb_init.shape[0]-1

        with tf.variable_scope('Gen_Embeddings_{}'.format(id)):

            self.u_node_embed  = tf.get_variable(name='u_node_embed', shape= node_emb_init.shape, 
                                              initializer=tf.constant_initializer(node_emb_init), trainable=True)
            
            self.v_node_embed  = tf.get_variable(name='v_node_embed', shape= node_emb_init.shape, 
                                              initializer=tf.constant_initializer(node_emb_init), trainable=True)
            
            self.topic_embed  = tf.get_variable(name='node_topic_embed', shape= topic_emb_init.shape, 
                                              initializer=tf.constant_initializer(topic_emb_init), trainable=True)
            
            
        with tf.name_scope('Gen_Input'):
            self.target_node = tf.placeholder(tf.int32, shape=[None, self.topic_num], name='target')
            self.neg_topic = tf.placeholder(tf.int32, shape=[None, self.topic_num], name= 'neg_topic')
            self.reward = tf.placeholder(tf.float32, shape=[None], name='reward')
            self.hard_neg_ind = tf.placeholder(tf.int32, shape=[None], name='hard_neg_index')
            
        with tf.name_scope('Embedding_Lookup'):
            self.target_emb = tf.nn.embedding_lookup(self.u_node_embed, self.target_node)
            self.neg_topic_emb = tf.nn.embedding_lookup(self.topic_embed, self.neg_topic)
            
        with tf.name_scope('Gen_Pro'):
            inner_pro = tf.reduce_sum(tf.multiply(self.target_emb, self.neg_topic_emb), -1)
            self.topic_inner_prob = tf.nn.softmax(tf.sigmoid(inner_pro))
            
        with tf.name_scope('Gen_Loss'):
            inner_pro = tf.reduce_sum(-1.0 * tf.multiply(self.target_emb, self.neg_topic_emb), -1)
            self.prob = tf.nn.softmax(tf.sigmoid(inner_pro))
            
            hard_neg_prob = tf.gather(tf.reshape(self.prob, [-1]), self.hard_neg_ind)
            #self.gen_loss = - tf.reduce_mean(tf.log(hard_neg_prob) * self.reward)
            #self.gen_loss = - tf.reduce_mean(tf.log(hard_neg_prob) )
            
            self.select_node_emb = tf.gather(tf.reshape(self.target_emb, [-1]), self.hard_neg_ind)
            self.gen_loss = - tf.reduce_mean(tf.log(hard_neg_prob) * self.reward) + 1e-5 * (tf.nn.l2_loss(self.target_emb)+tf.nn.l2_loss(self.neg_topic_emb))
            
            
            
            
#             self.pos_score = tf.reduce_sum(self.target_emb * self.pos_emb, axis=1)
#             self.pos_prob = tf.clip_by_value(tf.nn.sigmoid(self.pos_score), 1e-5, 1)
            
#             self.neg_score = tf.reduce_sum(-1.0 * self.target_emb * self.neg_emb, axis=1) 
#             self.neg_prob = tf.clip_by_value(tf.nn.sigmoid(self.neg_score), 1e-5, 1)
            
#             self.target_topic_score = tf.reduce_sum(self.target_emb * self.target_topic_emb, axis=1)
#             self.target_topic_prob = tf.clip_by_value(tf.nn.sigmoid(self.target_topic_score), 1e-5, 1)
            
#             self.neg_topic_score = tf.reduce_sum(-1.0 * self.target_emb * self.neg_topic_emb, axis=1)
#             self.neg_topic_prob = tf.clip_by_value(tf.nn.sigmoid(self.neg_topic_score), 1e-5, 1)
            
#             self.dis_loss = -tf.reduce_mean(tf.log(self.neg_topic_prob))-tf.reduce_mean(tf.log(self.pos_prob))-tf.reduce_mean(tf.log(self.neg_prob))-tf.reduce_mean(tf.log(self.target_topic_prob)) + 1e-5 * (tf.nn.l2_loss(self.neg_topic_emb)+tf.nn.l2_loss(self.target_emb)+tf.nn.l2_loss(self.pos_emb) + tf.nn.l2_loss(self.neg_emb)+tf.nn.l2_loss(self.target_topic_emb))

        with tf.name_scope('Gen_Train'):
            optimizer = tf.train.AdamOptimizer(lr_gen)
            self.gen_update = optimizer.minimize(self.gen_loss)

In [27]:
node_num = network.shape[0]
dim = 300
MIN = -6/np.sqrt(dim)
MAX = 6/np.sqrt(dim)
node_emb_init = (MAX-MIN) * np.random.random((node_num, dim)) + MIN

topic_num = norw_W.shape[1]
topic_dim = dim
MIN = -6/np.sqrt(topic_dim)
MAX = 6/np.sqrt(topic_dim)
topic_emb_init = (MAX-MIN) * np.random.random((topic_num, topic_dim)) + MIN

In [28]:
def gen_cal_inner_topic(topic_num,pos_batch):
    re_gen_pos_topic = []
    re_gen_neg_topic = []
    index = 0
    for t in pos_batch[:,2]:
        re_gen_pos_topic.append((topic_num-1)*[pos_batch[:,0][index]])
        tmp = []
        for i in range(topic_num):
            if i!=t:
                tmp.append(i)
        re_gen_neg_topic.append(tmp)
        index += 1
        #print("t ",t," ",tmp)
    return np.array(re_gen_pos_topic),np.array(re_gen_neg_topic)

In [31]:
tf.reset_default_graph()
sess.close()
lr_dis = 1e-3
lr_gen = 1e-3
discriminator = Discriminator(n_node=node_num, node_emd_init=node_emb_init, topic_emb_init=topic_emb_init, lr_dis=lr_dis, id = 0)
generator = Generator(n_node=node_num, node_emd_init=node_emb_init, topic_emb_init=topic_emb_init, lr_gen=lr_gen, id = 0)

tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True
init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
sess = tf.Session(config=tf_config)
sess.run(init_op)    

edge_num = len(edges)
batch_size = 1024
iters = int(edge_num/batch_size)
random_seq = np.random.permutation(edge_num)

reward_avg = 0
topic_num = W.shape[1]
range_topic_num = range(topic_num-1)

re_len = len(range_topic_num)

for i in range(iters):
    if i % 3000==0:
        print('start ',i,' out of ',iters)
    if i!=iters-1:
        index = random_seq[i*batch_size:(i+1)*batch_size]
    else:
        index = random_seq[i*batch_size:]
    
    pos_batch = edges[index]
    neg_batch = neg_pairs[index]
    
    
    pos_node_cand, neg_topic = gen_cal_inner_topic(topic_num,pos_batch)
    gen_prob = sess.run(generator.topic_inner_prob,
              feed_dict={generator.target_node: pos_node_cand,
                         generator.neg_topic: neg_topic}) 
    
#     select_neg_topic = []
#     for g_i in gen_prob:
#         select_neg_topic.append(np.random.choice(range_topic_num, p = g_i))
#     select_neg_topic = np.array(select_neg_topic)   
    
    select_neg_topic = []
    for g_i,n_i in zip(gen_prob,neg_topic):
        select_neg_topic.append(np.random.choice(n_i, p = g_i))
    select_neg_topic = np.array(select_neg_topic)   
    
    _, dis_loss = sess.run([discriminator.dis_update, discriminator.dis_loss],
              feed_dict={discriminator.target_node: pos_batch[:,0],
                         discriminator.pos_node: pos_batch[:,1],
                         discriminator.neg_node: neg_batch[:,1],
                         discriminator.target_topic: pos_batch[:,2],
                         discriminator.neg_topic: select_neg_topic})   
    
    reward = sess.run(discriminator.reward, {discriminator.target_node: pos_batch[:,0], discriminator.neg_topic: select_neg_topic})
    #reward = reward + reward_avg
    
    re_select_neg_topic = []
    for re_i in range(gen_prob.shape[0]):
        re_select_neg_topic.append(select_neg_topic[re_i]+re_i*re_len)
    re_select_neg_topic = np.array(re_select_neg_topic)
    
    
    _, gen_loss = sess.run([generator.gen_update,generator.gen_loss],
              feed_dict={generator.target_node: pos_node_cand,
                         generator.reward: reward,
                         generator.neg_topic: neg_topic,
                         generator.hard_neg_ind: re_select_neg_topic})  
    
    #print("np.sum(reward) / batch_size ",np.sum(reward) / batch_size)
    
    #reward_avg = np.sum(reward) / ((i+1)*batch_size) + reward_avg
#     sess.run(discriminator.dis_update,
#                   feed_dict={discriminator.target_node: pos_batch[:,0],
#                              discriminator.pos_node: pos_batch[:,1],
#                              discriminator.neg_node: neg_batch[:,1],
#                              discriminator.target_topic: pos_batch[:,2]})   
    
#     sess.run(discriminator.dis_update,
#                   feed_dict={discriminator.target_node: pos_batch[:,0],
#                              discriminator.pos_node: pos_batch[:,1],
#                              discriminator.neg_node: neg_batch[:,1]})   


    #print(type(dis_loss))
    #print("dis_loss: ",dis_loss," gen_loss: ",gen_loss," reward_avg: ",reward_avg," reward: ",np.sum(reward)/ batch_size)
    if i % 200==0:
        print("dis_loss: ",dis_loss," gen_loss: ",gen_loss," reward: ",np.sum(reward)/ batch_size)
            
    if i % 3000==0:
      
        vectors = sess.run(discriminator.u_node_embed)
        #vectors = vectors/np.sqrt(np.sum(np.square(vectors), -1)).reshape((vectors.shape[0],1))
        X, Y = read_node_label(label_file)

        print(" contex embedding.... ")
        import sys
        if not sys.warnoptions:
            import warnings
            warnings.simplefilter("ignore")
        avg_value = []
        for i in np.arange(0.1, 1.0, 0.1):
        #for i in np.arange(0.01, 0.11, 0.01):
            clf_ratio = np.round(i,2) 
            clf = Classifier(vectors=vectors, clf=LogisticRegression())
            avg_value.append(clf.split_train_evaluate(X, Y, clf_ratio)['micro'])
            print( clf_ratio," ",clf.split_train_evaluate(X, Y, clf_ratio)['micro'] )
        print(np.mean(avg_value))
        
        vectors = vectors/np.sqrt(np.sum(np.square(vectors), -1)).reshape((vectors.shape[0],1))
        print(" norm contex embedding.... ")
        avg_value = []
        for i in np.arange(0.1, 1.0, 0.1):
        #for i in np.arange(0.01, 0.11, 0.01):
            clf_ratio = np.round(i,2) 
            clf = Classifier(vectors=vectors, clf=LogisticRegression())
            avg_value.append(clf.split_train_evaluate(X, Y, clf_ratio)['micro'])
            print( clf_ratio," ",clf.split_train_evaluate(X, Y, clf_ratio)['micro'] )
        print(np.mean(avg_value))
        
        vectors = np.zeros((node_num,dim*2))
        left_vectors = sess.run(discriminator.u_node_embed)
        right_vectors = sess.run(discriminator.topic_embed)
        for i in range(left_vectors.shape[0]):
            vectors[i] = np.stack((left_vectors[i],right_vectors[topK_node[i]]),axis = 1).reshape(1,dim*2)
        #vectors = vectors/np.sqrt(np.sum(np.square(vectors), -1)).reshape((vectors.shape[0],1))
        print(" concate node and topic emb contex embedding.... ")
        avg_value = []
        for i in np.arange(0.1, 1.0, 0.1):
        #for i in np.arange(0.01, 0.11, 0.01):
            clf_ratio = np.round(i,2) 
            clf = Classifier(vectors=vectors, clf=LogisticRegression())
            avg_value.append(clf.split_train_evaluate(X, Y, clf_ratio)['micro'])
            print( clf_ratio," ",clf.split_train_evaluate(X, Y, clf_ratio)['micro'] )
        print(np.mean(avg_value))
        
        vectors = np.zeros((node_num,dim*2))
        left_vectors = sess.run(discriminator.u_node_embed)
        right_vectors = sess.run(discriminator.topic_embed)
        for i in range(left_vectors.shape[0]):
            vectors[i] = np.stack((left_vectors[i],right_vectors[topK_node[i]]),axis = 1).reshape(1,dim*2)
        vectors = vectors/np.sqrt(np.sum(np.square(vectors), -1)).reshape((vectors.shape[0],1))    
        #vectors = vectors/np.sqrt(np.sum(np.square(vectors), -1)).reshape((vectors.shape[0],1))
        print(" concate node and topic emb contex embedding.... ")
        avg_value = []
        for i in np.arange(0.1, 1.0, 0.1):
        #for i in np.arange(0.01, 0.11, 0.01):
            clf_ratio = np.round(i,2) 
            clf = Classifier(vectors=vectors, clf=LogisticRegression())
            avg_value.append(clf.split_train_evaluate(X, Y, clf_ratio)['micro'])
            print( clf_ratio," ",clf.split_train_evaluate(X, Y, clf_ratio)['micro'] )
        print(np.mean(avg_value))        
      
        print()

start  0  out of  39667
dis_loss:  3.2120001  gen_loss:  2.1820571  reward:  0.7824916839599609
 contex embedding.... 
0.1   0.2018047579983593
0.2   0.19796954314720813
0.3   0.20464135021097046
0.4   0.18584615384615386
0.5   0.19497784342688332
0.6   0.18726937269372693
0.7   0.1845018450184502
0.8   0.19557195571955718
0.9   0.1992619926199262
0.19464942385347062
 norm contex embedding.... 
0.1   0.29532403609515995
0.2   0.26903553299492383
0.3   0.26951476793248946
0.4   0.2713846153846154
0.5   0.25775480059084194
0.6   0.2555350553505535
0.7   0.25584255842558423
0.8   0.2564575645756458
0.9   0.23247232472324722
0.26259125067478467
 concate node and topic emb contex embedding.... 
0.1   0.38884331419196055
0.2   0.3544070143054915
0.3   0.34810126582278483
0.4   0.3495384615384615
0.5   0.3552437223042835
0.6   0.3422509225092251
0.7   0.3456334563345633
0.8   0.34317343173431736
0.9   0.3284132841328413
0.35062276365265876
 concate node and topic emb contex embedding.... 
0.1

dis_loss:  0.5645516  gen_loss:  inf  reward:  0.010653974488377571
dis_loss:  0.5374558  gen_loss:  0.018644601  reward:  0.010405750013887882
dis_loss:  0.5270932  gen_loss:  0.019915449  reward:  0.01111502293497324
dis_loss:  0.54855716  gen_loss:  0.01929217  reward:  0.010767164640128613
dis_loss:  0.54127663  gen_loss:  0.019696224  reward:  0.010992671363055706
dis_loss:  0.5301165  gen_loss:  0.01951282  reward:  0.010890310630202293
dis_loss:  0.5364665  gen_loss:  0.019365346  reward:  0.010808001272380352
dis_loss:  0.55017126  gen_loss:  0.01884949  reward:  0.010520100593566895
dis_loss:  0.53897274  gen_loss:  0.019101586  reward:  0.01066079642623663
dis_loss:  0.54615384  gen_loss:  0.019174729  reward:  0.0107016172260046
dis_loss:  0.5213915  gen_loss:  0.018379014  reward:  0.010257521644234657
dis_loss:  0.5315642  gen_loss:  0.018869108  reward:  0.01053104642778635
start  12000  out of  39667
dis_loss:  0.53023994  gen_loss:  0.019508464  reward:  0.0108878780156

0.4   0.796923076923077
0.5   0.809453471196455
0.6   0.8154981549815498
0.7   0.8290282902829028
0.8   0.8136531365313654
0.9   0.8118081180811808
0.805528565358621
 concate node and topic emb contex embedding.... 
0.1   0.7793273174733388
0.2   0.811259806183664
0.3   0.8222573839662448
0.4   0.8258461538461539
0.5   0.8382570162481536
0.6   0.8496309963099631
0.7   0.8646986469864698
0.8   0.8597785977859779
0.9   0.8487084870848709
0.8333071562094262
 concate node and topic emb contex embedding.... 
0.1   0.7547169811320754
0.2   0.792339640055376
0.3   0.8080168776371308
0.4   0.792
0.5   0.8057607090103397
0.6   0.8071955719557196
0.7   0.8216482164821647
0.8   0.8062730627306273
0.9   0.8154981549815498
0.8003832459983315

dis_loss:  0.52822876  gen_loss:  0.018884016  reward:  0.010538586415350437
dis_loss:  0.5512799  gen_loss:  0.018825151  reward:  0.010506028309464455
dis_loss:  0.53296006  gen_loss:  0.018688712  reward:  0.010429756715893745
dis_loss:  0.5324656  gen_loss

dis_loss:  0.5594995  gen_loss:  0.019383587  reward:  0.01081762183457613
dis_loss:  0.51617074  gen_loss:  0.019157087  reward:  0.010691039264202118
dis_loss:  0.51657254  gen_loss:  0.018845385  reward:  0.01051718182861805
dis_loss:  0.5480456  gen_loss:  0.019018603  reward:  0.010613917373120785
dis_loss:  0.5599418  gen_loss:  0.019199546  reward:  0.010714812204241753
start  33000  out of  39667
dis_loss:  0.571246  gen_loss:  0.019539097  reward:  0.010904650203883648
 contex embedding.... 
0.1   0.780557834290402
0.2   0.8149515459160129
0.3   0.819620253164557
0.4   0.8264615384615385
0.5   0.8426883308714919
0.6   0.8422509225092251
0.7   0.8560885608856088
0.8   0.8468634686346863
0.9   0.8523985239852399
0.8313201087465292
 norm contex embedding.... 
0.1   0.7649712879409352
0.2   0.7951084448546377
0.3   0.8085443037974683
0.4   0.7963076923076923
0.5   0.808714918759232
0.6   0.8173431734317345
0.7   0.8351783517835178
0.8   0.8210332103321034
0.9   0.8265682656826568


In [32]:
        vectors = sess.run(discriminator.u_node_embed)
        #vectors = vectors/np.sqrt(np.sum(np.square(vectors), -1)).reshape((vectors.shape[0],1))
        X, Y = read_node_label(label_file)

        print(" contex embedding.... ")
        import sys
        if not sys.warnoptions:
            import warnings
            warnings.simplefilter("ignore")
        avg_value = []
        for i in np.arange(0.1, 1.0, 0.1):
        #for i in np.arange(0.01, 0.11, 0.01):
            clf_ratio = np.round(i,2) 
            clf = Classifier(vectors=vectors, clf=LogisticRegression())
            avg_value.append(clf.split_train_evaluate(X, Y, clf_ratio)['micro'])
            print( clf_ratio," ",clf.split_train_evaluate(X, Y, clf_ratio)['micro'] )
        print(np.mean(avg_value))
        
        vectors = vectors/np.sqrt(np.sum(np.square(vectors), -1)).reshape((vectors.shape[0],1))
        print(" norm contex embedding.... ")
        avg_value = []
        for i in np.arange(0.1, 1.0, 0.1):
        #for i in np.arange(0.01, 0.11, 0.01):
            clf_ratio = np.round(i,2) 
            clf = Classifier(vectors=vectors, clf=LogisticRegression())
            avg_value.append(clf.split_train_evaluate(X, Y, clf_ratio)['micro'])
            print( clf_ratio," ",clf.split_train_evaluate(X, Y, clf_ratio)['micro'] )
        print(np.mean(avg_value))
        
        vectors = np.zeros((node_num,dim*2))
        left_vectors = sess.run(discriminator.u_node_embed)
        right_vectors = sess.run(discriminator.topic_embed)
        for i in range(left_vectors.shape[0]):
            vectors[i] = np.stack((left_vectors[i],right_vectors[topK_node[i]]),axis = 1).reshape(1,dim*2)
        #vectors = vectors/np.sqrt(np.sum(np.square(vectors), -1)).reshape((vectors.shape[0],1))
        print(" concate node and topic emb contex embedding.... ")
        avg_value = []
        for i in np.arange(0.1, 1.0, 0.1):
        #for i in np.arange(0.01, 0.11, 0.01):
            clf_ratio = np.round(i,2) 
            clf = Classifier(vectors=vectors, clf=LogisticRegression())
            avg_value.append(clf.split_train_evaluate(X, Y, clf_ratio)['micro'])
            print( clf_ratio," ",clf.split_train_evaluate(X, Y, clf_ratio)['micro'] )
        print(np.mean(avg_value))
        
        vectors = np.zeros((node_num,dim*2))
        left_vectors = sess.run(discriminator.u_node_embed)
        right_vectors = sess.run(discriminator.topic_embed)
        for i in range(left_vectors.shape[0]):
            vectors[i] = np.stack((left_vectors[i],right_vectors[topK_node[i]]),axis = 1).reshape(1,dim*2)
        vectors = vectors/np.sqrt(np.sum(np.square(vectors), -1)).reshape((vectors.shape[0],1))    
        #vectors = vectors/np.sqrt(np.sum(np.square(vectors), -1)).reshape((vectors.shape[0],1))
        print(" concate node and topic emb contex embedding.... ")
        avg_value = []
        for i in np.arange(0.1, 1.0, 0.1):
        #for i in np.arange(0.01, 0.11, 0.01):
            clf_ratio = np.round(i,2) 
            clf = Classifier(vectors=vectors, clf=LogisticRegression())
            avg_value.append(clf.split_train_evaluate(X, Y, clf_ratio)['micro'])
            print( clf_ratio," ",clf.split_train_evaluate(X, Y, clf_ratio)['micro'] )
        print(np.mean(avg_value))        
      
        print()

 contex embedding.... 
0.1   0.785890073831009
0.2   0.8177203507152746
0.3   0.8285864978902954
0.4   0.8326153846153846
0.5   0.846381093057607
0.6   0.8496309963099631
0.7   0.8548585485854858
0.8   0.8542435424354243
0.9   0.8745387453874539
0.8382739147586553
 norm contex embedding.... 
0.1   0.7694831829368335
0.2   0.8011075219197047
0.3   0.8106540084388185
0.4   0.798153846153846
0.5   0.8168389955686853
0.6   0.8228782287822878
0.7   0.8364083640836408
0.8   0.8247232472324724
0.9   0.8302583025830258
0.8122784108554794
 concate node and topic emb contex embedding.... 
0.1   0.7883511074651354
0.2   0.8200276880479926
0.3   0.8296413502109705
0.4   0.8301538461538461
0.5   0.8419497784342688
0.6   0.8440959409594095
0.7   0.8523985239852399
0.8   0.8487084870848709
0.9   0.8708487084870848
0.8362417145365354
 concate node and topic emb contex embedding.... 
0.1   0.7596390484003281
0.2   0.7941855099215506
0.3   0.80168776371308
0.4   0.7944615384615384
0.5   0.80649926144756

In [33]:
        vectors = sess.run(discriminator.u_node_embed)
        #vectors = vectors/np.sqrt(np.sum(np.square(vectors), -1)).reshape((vectors.shape[0],1))
        X, Y = read_node_label(label_file)

        print(" contex embedding.... ")
        import sys
        if not sys.warnoptions:
            import warnings
            warnings.simplefilter("ignore")
        avg_value = []
        for i in np.arange(0.1, 1.0, 0.1):
        #for i in np.arange(0.01, 0.11, 0.01):
            clf_ratio = np.round(i,2) 
            clf = Classifier(vectors=vectors, clf=LogisticRegression())
            avg_value.append(clf.split_train_evaluate(X, Y, clf_ratio)['micro'])
            print( clf_ratio," ",clf.split_train_evaluate(X, Y, clf_ratio)['micro'] )
        print(np.mean(avg_value))
        
        vectors = vectors/np.sqrt(np.sum(np.square(vectors), -1)).reshape((vectors.shape[0],1))
        print(" norm contex embedding.... ")
        avg_value = []
        for i in np.arange(0.1, 1.0, 0.1):
        #for i in np.arange(0.01, 0.11, 0.01):
            clf_ratio = np.round(i,2) 
            clf = Classifier(vectors=vectors, clf=LogisticRegression())
            avg_value.append(clf.split_train_evaluate(X, Y, clf_ratio)['micro'])
            print( clf_ratio," ",clf.split_train_evaluate(X, Y, clf_ratio)['micro'] )
        print(np.mean(avg_value))
        
        vectors = np.zeros((node_num,dim*2))
        left_vectors = sess.run(discriminator.u_node_embed)
        right_vectors = sess.run(discriminator.topic_embed)
        for i in range(left_vectors.shape[0]):
            vectors[i] = np.stack((left_vectors[i],right_vectors[topK_node[i]]),axis = 1).reshape(1,dim*2)
        #vectors = vectors/np.sqrt(np.sum(np.square(vectors), -1)).reshape((vectors.shape[0],1))
        print(" concate node and topic emb contex embedding.... ")
        avg_value = []
        for i in np.arange(0.1, 1.0, 0.1):
        #for i in np.arange(0.01, 0.11, 0.01):
            clf_ratio = np.round(i,2) 
            clf = Classifier(vectors=vectors, clf=LogisticRegression())
            avg_value.append(clf.split_train_evaluate(X, Y, clf_ratio)['micro'])
            print( clf_ratio," ",clf.split_train_evaluate(X, Y, clf_ratio)['micro'] )
        print(np.mean(avg_value))
        
        vectors = np.zeros((node_num,dim*2))
        left_vectors = sess.run(discriminator.u_node_embed)
        right_vectors = sess.run(discriminator.topic_embed)
        for i in range(left_vectors.shape[0]):
            vectors[i] = np.stack((left_vectors[i],right_vectors[topK_node[i]]),axis = 1).reshape(1,dim*2)
        vectors = vectors/np.sqrt(np.sum(np.square(vectors), -1)).reshape((vectors.shape[0],1))    
        #vectors = vectors/np.sqrt(np.sum(np.square(vectors), -1)).reshape((vectors.shape[0],1))
        print(" concate node and topic emb contex embedding.... ")
        avg_value = []
        for i in np.arange(0.1, 1.0, 0.1):
        #for i in np.arange(0.01, 0.11, 0.01):
            clf_ratio = np.round(i,2) 
            clf = Classifier(vectors=vectors, clf=LogisticRegression())
            avg_value.append(clf.split_train_evaluate(X, Y, clf_ratio)['micro'])
            print( clf_ratio," ",clf.split_train_evaluate(X, Y, clf_ratio)['micro'] )
        print(np.mean(avg_value))        
      
        print()

 contex embedding.... 
0.1   0.785890073831009
0.2   0.8177203507152746
0.3   0.8285864978902954
0.4   0.8326153846153846
0.5   0.846381093057607
0.6   0.8496309963099631
0.7   0.8548585485854858
0.8   0.8542435424354243
0.9   0.8745387453874539
0.8382739147586553
 norm contex embedding.... 
0.1   0.7694831829368335
0.2   0.8011075219197047
0.3   0.8106540084388185
0.4   0.798153846153846
0.5   0.8168389955686853
0.6   0.8228782287822878
0.7   0.8364083640836408
0.8   0.8247232472324724
0.9   0.8302583025830258
0.8122784108554794
 concate node and topic emb contex embedding.... 
0.1   0.7883511074651354
0.2   0.8200276880479926
0.3   0.8296413502109705
0.4   0.8301538461538461
0.5   0.8419497784342688
0.6   0.8440959409594095
0.7   0.8523985239852399
0.8   0.8487084870848709
0.9   0.8708487084870848
0.8362417145365354
 concate node and topic emb contex embedding.... 
0.1   0.7596390484003281
0.2   0.7941855099215506
0.3   0.80168776371308
0.4   0.7944615384615384
0.5   0.80649926144756

In [43]:
        vectors = sess.run(discriminator.u_node_embed)
        #vectors = vectors/np.sqrt(np.sum(np.square(vectors), -1)).reshape((vectors.shape[0],1))
        X, Y = read_node_label(label_file)

        print(" contex embedding.... ")
        import sys
        if not sys.warnoptions:
            import warnings
            warnings.simplefilter("ignore")
        avg_value = []
        for i in np.arange(0.1, 1.0, 0.1):
        #for i in np.arange(0.01, 0.11, 0.01):
            clf_ratio = np.round(i,2) 
            clf = Classifier(vectors=vectors, clf=LogisticRegression())
            avg_value.append(clf.split_train_evaluate(X, Y, clf_ratio)['micro'])
            print( clf_ratio," ",clf.split_train_evaluate(X, Y, clf_ratio)['micro'] )
        print(np.mean(avg_value))
        
        vectors = vectors/np.sqrt(np.sum(np.square(vectors), -1)).reshape((vectors.shape[0],1))
        print(" norm contex embedding.... ")
        avg_value = []
        for i in np.arange(0.1, 1.0, 0.1):
        #for i in np.arange(0.01, 0.11, 0.01):
            clf_ratio = np.round(i,2) 
            clf = Classifier(vectors=vectors, clf=LogisticRegression())
            avg_value.append(clf.split_train_evaluate(X, Y, clf_ratio)['micro'])
            print( clf_ratio," ",clf.split_train_evaluate(X, Y, clf_ratio)['micro'] )
        print(np.mean(avg_value))
        
        vectors = np.zeros((node_num,dim*2))
        left_vectors = sess.run(discriminator.u_node_embed)
        right_vectors = sess.run(discriminator.topic_embed)
        for i in range(left_vectors.shape[0]):
            vectors[i] = np.stack((left_vectors[i],right_vectors[topK_node[i]]),axis = 1).reshape(1,dim*2)
        #vectors = vectors/np.sqrt(np.sum(np.square(vectors), -1)).reshape((vectors.shape[0],1))
        print(" concate node and topic emb contex embedding.... ")
        avg_value = []
        for i in np.arange(0.1, 1.0, 0.1):
        #for i in np.arange(0.01, 0.11, 0.01):
            clf_ratio = np.round(i,2) 
            clf = Classifier(vectors=vectors, clf=LogisticRegression())
            avg_value.append(clf.split_train_evaluate(X, Y, clf_ratio)['micro'])
            print( clf_ratio," ",clf.split_train_evaluate(X, Y, clf_ratio)['micro'] )
        print(np.mean(avg_value))
        
        vectors = np.zeros((node_num,dim*2))
        left_vectors = sess.run(discriminator.u_node_embed)
        right_vectors = sess.run(discriminator.topic_embed)
        for i in range(left_vectors.shape[0]):
            vectors[i] = np.stack((left_vectors[i],right_vectors[topK_node[i]]),axis = 1).reshape(1,dim*2)
        vectors = vectors/np.sqrt(np.sum(np.square(vectors), -1)).reshape((vectors.shape[0],1))    
        #vectors = vectors/np.sqrt(np.sum(np.square(vectors), -1)).reshape((vectors.shape[0],1))
        print(" concate node and topic emb contex embedding.... ")
        avg_value = []
        for i in np.arange(0.1, 1.0, 0.1):
        #for i in np.arange(0.01, 0.11, 0.01):
            clf_ratio = np.round(i,2) 
            clf = Classifier(vectors=vectors, clf=LogisticRegression())
            avg_value.append(clf.split_train_evaluate(X, Y, clf_ratio)['micro'])
            print( clf_ratio," ",clf.split_train_evaluate(X, Y, clf_ratio)['micro'] )
        print(np.mean(avg_value))        
      
        print()

 contex embedding.... 
0.1   0.7748154224774406
0.2   0.810336871250577
0.3   0.8175105485232067
0.4   0.8233846153846154
0.5   0.8345642540620384
0.6   0.8431734317343174
0.7   0.8511685116851169
0.8   0.8468634686346863
0.9   0.8597785977859779
0.8290661912819973
 norm contex embedding.... 
0.1   0.7662018047579984
0.2   0.7941855099215506
0.3   0.805379746835443
0.4   0.7956923076923077
0.5   0.8124076809453471
0.6   0.8136531365313654
0.7   0.8290282902829028
0.8   0.8081180811808119
0.9   0.8007380073800738
0.8028227295030891
 concate node and topic emb contex embedding.... 
0.1   0.7764561115668581
0.2   0.8121827411167513
0.3   0.8164556962025317
0.4   0.8178461538461539
0.5   0.8330871491875923
0.6   0.8394833948339483
0.7   0.8487084870848709
0.8   0.8431734317343174
0.9   0.8560885608856088
0.8270535251620703
 concate node and topic emb contex embedding.... 
0.1   0.7551271534044298
0.2   0.7844946931241348
0.3   0.8032700421940928
0.4   0.7950769230769231
0.5   0.80797636632

In [38]:
        vectors = sess.run(discriminator.u_node_embed)
        #vectors = vectors/np.sqrt(np.sum(np.square(vectors), -1)).reshape((vectors.shape[0],1))
        X, Y = read_node_label(label_file)

        print(" contex embedding.... ")
        import sys
        if not sys.warnoptions:
            import warnings
            warnings.simplefilter("ignore")
        avg_value = []
        for i in np.arange(0.1, 1.0, 0.1):
        #for i in np.arange(0.01, 0.11, 0.01):
            clf_ratio = np.round(i,2) 
            clf = Classifier(vectors=vectors, clf=LogisticRegression())
            avg_value.append(clf.split_train_evaluate(X, Y, clf_ratio)['micro'])
            print( clf_ratio," ",clf.split_train_evaluate(X, Y, clf_ratio)['micro'] )
        print(np.mean(avg_value))
        
        vectors = vectors/np.sqrt(np.sum(np.square(vectors), -1)).reshape((vectors.shape[0],1))
        print(" norm contex embedding.... ")
        avg_value = []
        for i in np.arange(0.1, 1.0, 0.1):
        #for i in np.arange(0.01, 0.11, 0.01):
            clf_ratio = np.round(i,2) 
            clf = Classifier(vectors=vectors, clf=LogisticRegression())
            avg_value.append(clf.split_train_evaluate(X, Y, clf_ratio)['micro'])
            print( clf_ratio," ",clf.split_train_evaluate(X, Y, clf_ratio)['micro'] )
        print(np.mean(avg_value))
        
        vectors = np.zeros((node_num,dim*2))
        left_vectors = sess.run(discriminator.u_node_embed)
        right_vectors = sess.run(discriminator.topic_embed)
        for i in range(left_vectors.shape[0]):
            vectors[i] = np.stack((left_vectors[i],right_vectors[topK_node[i]]),axis = 1).reshape(1,dim*2)
        #vectors = vectors/np.sqrt(np.sum(np.square(vectors), -1)).reshape((vectors.shape[0],1))
        print(" concate node and topic emb contex embedding.... ")
        avg_value = []
        for i in np.arange(0.1, 1.0, 0.1):
        #for i in np.arange(0.01, 0.11, 0.01):
            clf_ratio = np.round(i,2) 
            clf = Classifier(vectors=vectors, clf=LogisticRegression())
            avg_value.append(clf.split_train_evaluate(X, Y, clf_ratio)['micro'])
            print( clf_ratio," ",clf.split_train_evaluate(X, Y, clf_ratio)['micro'] )
        print(np.mean(avg_value))
        
        vectors = np.zeros((node_num,dim*2))
        left_vectors = sess.run(discriminator.u_node_embed)
        right_vectors = sess.run(discriminator.topic_embed)
        for i in range(left_vectors.shape[0]):
            vectors[i] = np.stack((left_vectors[i],right_vectors[topK_node[i]]),axis = 1).reshape(1,dim*2)
        vectors = vectors/np.sqrt(np.sum(np.square(vectors), -1)).reshape((vectors.shape[0],1))    
        #vectors = vectors/np.sqrt(np.sum(np.square(vectors), -1)).reshape((vectors.shape[0],1))
        print(" concate node and topic emb contex embedding.... ")
        avg_value = []
        for i in np.arange(0.1, 1.0, 0.1):
        #for i in np.arange(0.01, 0.11, 0.01):
            clf_ratio = np.round(i,2) 
            clf = Classifier(vectors=vectors, clf=LogisticRegression())
            avg_value.append(clf.split_train_evaluate(X, Y, clf_ratio)['micro'])
            print( clf_ratio," ",clf.split_train_evaluate(X, Y, clf_ratio)['micro'] )
        print(np.mean(avg_value))        
      
        print()

 contex embedding.... 
0.1   0.7752255947497949
0.2   0.8154130133825565
0.3   0.82542194092827
0.4   0.8215384615384616
0.5   0.8345642540620384
0.6   0.8431734317343174
0.7   0.8511685116851169
0.8   0.8560885608856088
0.9   0.8708487084870848
0.8326047197170277
 norm contex embedding.... 
0.1   0.7698933552091878
0.2   0.796031379787725
0.3   0.810126582278481
0.4   0.803076923076923
0.5   0.8153618906942393
0.6   0.8154981549815498
0.7   0.8351783517835178
0.8   0.8228782287822878
0.9   0.8191881918819188
0.8096925620528701
 concate node and topic emb contex embedding.... 
0.1   0.7760459392945037
0.2   0.8181818181818182
0.3   0.8259493670886076
0.4   0.8172307692307692
0.5   0.8345642540620384
0.6   0.8404059040590407
0.7   0.8511685116851169
0.8   0.8560885608856088
0.9   0.8671586715867159
0.83186597734158
 concate node and topic emb contex embedding.... 
0.1   0.7608695652173914
0.2   0.7891093677895709
0.3   0.8048523206751055
0.4   0.7993846153846154
0.5   0.8116691285081241

In [80]:
        vectors = sess.run(discriminator.u_node_embed)
        #vectors = vectors/np.sqrt(np.sum(np.square(vectors), -1)).reshape((vectors.shape[0],1))
        X, Y = read_node_label(label_file)

        print(" contex embedding.... ")
        import sys
        if not sys.warnoptions:
            import warnings
            warnings.simplefilter("ignore")
        avg_value = []
        for i in np.arange(0.1, 1.0, 0.1):
        #for i in np.arange(0.01, 0.11, 0.01):
            clf_ratio = np.round(i,2) 
            clf = Classifier(vectors=vectors, clf=LogisticRegression())
            avg_value.append(clf.split_train_evaluate(X, Y, clf_ratio)['micro'])
            print( clf_ratio," ",clf.split_train_evaluate(X, Y, clf_ratio)['micro'] )
        print(np.mean(avg_value))
        
        vectors = vectors/np.sqrt(np.sum(np.square(vectors), -1)).reshape((vectors.shape[0],1))
        print(" norm contex embedding.... ")
        avg_value = []
        for i in np.arange(0.1, 1.0, 0.1):
        #for i in np.arange(0.01, 0.11, 0.01):
            clf_ratio = np.round(i,2) 
            clf = Classifier(vectors=vectors, clf=LogisticRegression())
            avg_value.append(clf.split_train_evaluate(X, Y, clf_ratio)['micro'])
            print( clf_ratio," ",clf.split_train_evaluate(X, Y, clf_ratio)['micro'] )
        print(np.mean(avg_value))
        
        vectors = np.zeros((node_num,dim*2))
        left_vectors = sess.run(discriminator.u_node_embed)
        right_vectors = sess.run(discriminator.topic_embed)
        for i in range(left_vectors.shape[0]):
            vectors[i] = np.stack((left_vectors[i],right_vectors[topK_node[i]]),axis = 1).reshape(1,dim*2)
        #vectors = vectors/np.sqrt(np.sum(np.square(vectors), -1)).reshape((vectors.shape[0],1))
        print(" concate node and topic emb contex embedding.... ")
        avg_value = []
        for i in np.arange(0.1, 1.0, 0.1):
        #for i in np.arange(0.01, 0.11, 0.01):
            clf_ratio = np.round(i,2) 
            clf = Classifier(vectors=vectors, clf=LogisticRegression())
            avg_value.append(clf.split_train_evaluate(X, Y, clf_ratio)['micro'])
            print( clf_ratio," ",clf.split_train_evaluate(X, Y, clf_ratio)['micro'] )
        print(np.mean(avg_value))
        
        vectors = np.zeros((node_num,dim*2))
        left_vectors = sess.run(discriminator.u_node_embed)
        right_vectors = sess.run(discriminator.topic_embed)
        for i in range(left_vectors.shape[0]):
            vectors[i] = np.stack((left_vectors[i],right_vectors[topK_node[i]]),axis = 1).reshape(1,dim*2)
        vectors = vectors/np.sqrt(np.sum(np.square(vectors), -1)).reshape((vectors.shape[0],1))    
        #vectors = vectors/np.sqrt(np.sum(np.square(vectors), -1)).reshape((vectors.shape[0],1))
        print(" concate node and topic emb contex embedding.... ")
        avg_value = []
        for i in np.arange(0.1, 1.0, 0.1):
        #for i in np.arange(0.01, 0.11, 0.01):
            clf_ratio = np.round(i,2) 
            clf = Classifier(vectors=vectors, clf=LogisticRegression())
            avg_value.append(clf.split_train_evaluate(X, Y, clf_ratio)['micro'])
            print( clf_ratio," ",clf.split_train_evaluate(X, Y, clf_ratio)['micro'] )
        print(np.mean(avg_value))        
      
        print()

 contex embedding.... 
0.1   0.7871205906480722
0.2   0.8181818181818182
0.3   0.8264767932489453
0.4   0.8178461538461539
0.5   0.8345642540620384
0.6   0.8459409594095942
0.7   0.8573185731857319
0.8   0.8560885608856088
0.9   0.8819188191881918
0.8361618358506839
 norm contex embedding.... 
0.1   0.773174733388023
0.2   0.7932625749884633
0.3   0.809071729957806
0.4   0.7956923076923077
0.5   0.810930576070901
0.6   0.8154981549815498
0.7   0.8339483394833949
0.8   0.8228782287822878
0.9   0.8228782287822878
0.8085927637918912
 concate node and topic emb contex embedding.... 
0.1   0.784659557013946
0.2   0.8186432856483619
0.3   0.8248945147679325
0.4   0.8221538461538461
0.5   0.8301329394387
0.6   0.8477859778597786
0.7   0.8585485854858549
0.8   0.8505535055350554
0.9   0.8671586715867159
0.8338367648322434
 concate node and topic emb contex embedding.... 
0.1   0.7584085315832649
0.2   0.7881864328564836
0.3   0.8074894514767933
0.4   0.7932307692307692
0.5   0.8057607090103397

In [125]:
topic_vectors = sess.run(discriminator.topic_embed)
u_vectors = sess.run(discriminator.u_node_embed)

import math
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

num_k = W.shape[1]
final_topic_assign = {}
for i in range(node_num):
    final_topic_assign[i] = None
    tmp = []
    for j in range(num_k):
        tmp.append(sigmoid(np.matmul(u_vectors[i],topic_vectors[j].T)))
    final_topic_assign[i] = tmp.index(max(tmp))

vectors = sess.run(discriminator.u_node_embed)
#vectors = vectors/np.sqrt(np.sum(np.square(vectors), -1)).reshape((vectors.shape[0],1))
X, Y = read_node_label(label_file)

print(" contex embedding.... ")
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
avg_value = []
for i in np.arange(0.1, 1.0, 0.1):
#for i in np.arange(0.01, 0.11, 0.01):
    clf_ratio = np.round(i,2) 
    clf = Classifier(vectors=vectors, clf=LogisticRegression())
    avg_value.append(clf.split_train_evaluate(X, Y, clf_ratio)['micro'])
    print( clf_ratio," ",clf.split_train_evaluate(X, Y, clf_ratio)['micro'] )
print(np.mean(avg_value))

vectors = vectors/np.sqrt(np.sum(np.square(vectors), -1)).reshape((vectors.shape[0],1))
print(" norm contex embedding.... ")
avg_value = []
for i in np.arange(0.1, 1.0, 0.1):
#for i in np.arange(0.01, 0.11, 0.01):
    clf_ratio = np.round(i,2) 
    clf = Classifier(vectors=vectors, clf=LogisticRegression())
    avg_value.append(clf.split_train_evaluate(X, Y, clf_ratio)['micro'])
    print( clf_ratio," ",clf.split_train_evaluate(X, Y, clf_ratio)['micro'] )
print(np.mean(avg_value))

vectors = np.zeros((node_num,dim*2))
left_vectors = sess.run(discriminator.u_node_embed)
right_vectors = sess.run(discriminator.topic_embed)
for i in range(left_vectors.shape[0]):
    vectors[i] = np.stack((left_vectors[i],right_vectors[final_topic_assign[i]]),axis = 1).reshape(1,dim*2)
#vectors = vectors/np.sqrt(np.sum(np.square(vectors), -1)).reshape((vectors.shape[0],1))
print(" concate node and topic emb contex embedding.... ")
avg_value = []
for i in np.arange(0.1, 1.0, 0.1):
#for i in np.arange(0.01, 0.11, 0.01):
    clf_ratio = np.round(i,2) 
    clf = Classifier(vectors=vectors, clf=LogisticRegression())
    avg_value.append(clf.split_train_evaluate(X, Y, clf_ratio)['micro'])
    print( clf_ratio," ",clf.split_train_evaluate(X, Y, clf_ratio)['micro'] )
print(np.mean(avg_value))

vectors = np.zeros((node_num,dim*2))
left_vectors = sess.run(discriminator.u_node_embed)
right_vectors = sess.run(discriminator.topic_embed)
for i in range(left_vectors.shape[0]):
    vectors[i] = np.stack((left_vectors[i],right_vectors[final_topic_assign[i]]),axis = 1).reshape(1,dim*2)
vectors = vectors/np.sqrt(np.sum(np.square(vectors), -1)).reshape((vectors.shape[0],1))    
#vectors = vectors/np.sqrt(np.sum(np.square(vectors), -1)).reshape((vectors.shape[0],1))
print(" concate node and topic emb contex embedding.... ")
avg_value = []
for i in np.arange(0.1, 1.0, 0.1):
#for i in np.arange(0.01, 0.11, 0.01):
    clf_ratio = np.round(i,2) 
    clf = Classifier(vectors=vectors, clf=LogisticRegression())
    avg_value.append(clf.split_train_evaluate(X, Y, clf_ratio)['micro'])
    print( clf_ratio," ",clf.split_train_evaluate(X, Y, clf_ratio)['micro'] )
print(np.mean(avg_value))        

print()

 contex embedding.... 
0.1   0.7871205906480722
0.2   0.8181818181818182
0.3   0.8264767932489453
0.4   0.8178461538461539
0.5   0.8345642540620384
0.6   0.8459409594095942
0.7   0.8573185731857319
0.8   0.8560885608856088
0.9   0.8819188191881918
0.8361618358506839
 norm contex embedding.... 
0.1   0.773174733388023
0.2   0.7932625749884633
0.3   0.809071729957806
0.4   0.7956923076923077
0.5   0.810930576070901
0.6   0.8154981549815498
0.7   0.8339483394833949
0.8   0.8228782287822878
0.9   0.8228782287822878
0.8085927637918912
 concate node and topic emb contex embedding.... 
0.1   0.785890073831009
0.2   0.8191047531149054
0.3   0.8270042194092827
0.4   0.8203076923076924
0.5   0.8338257016248154
0.6   0.8450184501845017
0.7   0.8573185731857319
0.8   0.8560885608856088
0.9   0.8708487084870848
0.8350451925589591
 concate node and topic emb contex embedding.... 
0.1   0.756357670221493
0.2   0.78772496538994
0.3   0.80168776371308
0.4   0.7926153846153846
0.5   0.8035450516986706
0

In [100]:
topic_vectors = sess.run(discriminator.topic_embed)
u_vectors = sess.run(discriminator.u_node_embed)

import math
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

num_k = W.shape[1]
final_topic_assign = {}
for i in range(node_num):
    final_topic_assign[i] = None
    tmp = []
    norm_tmp = []
    for j in range(num_k):
        thres_value = sigmoid(np.matmul(u_vectors[i],topic_vectors[j].T))
        #print(thres_value)
        #if thres_value >= 0.1:
            #tmp.append(j)
        tmp.append(thres_value)
    tmp_sum = sum(tmp)
    tmp = [i/tmp_sum for i in tmp]
    for tmp_i in range(num_k):
        if tmp[tmp_i] >= 0.15:
            norm_tmp.append(tmp_i)
    print(norm_tmp)
    final_topic_assign[i] = norm_tmp
    

degrees = list(np.reshape(np.array(np.sum(network, 1)), (network.shape[0],)))
total_link_num = 0
for d_i in degrees:
    total_link_num += d_i   

[4, 5]
[1, 3]
[1, 3]
[1, 3, 5]
[4, 5]
[0, 1, 3]
[1, 3]
[4, 5]
[4, 5]
[1, 3, 4]
[4, 5]
[0, 2]
[4, 5]
[0, 1, 2, 6]
[1, 4, 5]
[0, 1, 5]
[1, 3]
[4, 5]
[4, 5]
[1, 4, 5]
[1, 6]
[1]
[0, 2]
[4, 5]
[1, 3]
[1]
[4, 5]
[4, 5]
[1, 5]
[1, 2, 6]
[0, 2, 3, 4, 6]
[1, 4, 5]
[0, 2, 3, 4, 6]
[4, 5]
[1, 3]
[1, 3]
[4, 5]
[0, 2]
[0, 2]
[1, 3]
[1, 4]
[4, 5]
[0, 2]
[4, 5]
[1]
[1, 3]
[4, 5]
[1, 4, 5]
[3, 4, 5]
[0, 3, 4]
[0, 3, 6]
[4, 5]
[1]
[0, 2, 6]
[0, 1]
[0, 2]
[1]
[1, 3]
[4, 5]
[4, 5]
[1, 3]
[3, 4, 6]
[1, 3, 4]
[1, 3]
[0, 2, 3, 6]
[0, 2, 6]
[1, 3]
[3, 4]
[1, 4, 5]
[1, 3]
[1, 5]
[1, 5]
[4, 5]
[1, 5]
[1, 5]
[1, 3, 4]
[1, 4, 5]
[0, 2]
[1, 4, 5]
[1]
[5]
[3, 4]
[1, 3]
[1, 5]
[4, 5]
[1, 3]
[3, 4, 6]
[1, 5]
[1, 5]
[1]
[1, 4, 5]
[4, 5]
[1, 3]
[4, 5]
[3, 4, 5]
[4, 5]
[1]
[4]
[1, 3]
[4, 5]
[0, 2, 6]
[1, 4, 5]
[1]
[0, 1, 3]
[4, 5]
[3, 4]
[1, 5]
[1, 4, 5]
[1, 4, 5]
[1]
[4, 5]
[4, 5]
[0, 5]
[4, 5]
[1, 2]
[4, 5]
[1]
[4, 5]
[0, 1, 3]
[1, 2, 6]
[0, 2, 6]
[4, 5]
[4, 5]
[4, 5]
[4, 5]
[1, 4, 5]
[3, 4, 5]
[4, 5]
[0, 1, 3]
[0, 

[0, 2, 6]
[0, 1]
[1, 3]
[0, 2]
[1, 5]
[3, 4]
[1, 5]
[1, 4, 6]
[1, 3]
[0, 2]
[1, 4, 5, 6]
[1, 3]
[1, 6]
[0, 1, 3]
[1]
[3, 4, 5]
[1, 5]
[1, 3]
[1, 3]
[0, 2, 6]
[4, 5]
[4, 5]
[1, 5]
[1, 3]
[1, 4, 5]
[4, 5]
[4, 5]
[1, 3]
[1, 5]
[0, 2]
[0, 2]
[0, 2]
[0, 1, 2, 6]
[4]
[0, 2, 3, 4, 6]
[3, 4]
[4, 5]
[0, 2, 3, 6]
[1, 3]
[0, 3]
[0, 2, 3, 6]
[0, 2, 6]
[3, 4]
[0, 2]
[0, 2]
[1, 5]
[0, 2]
[3, 4, 5]
[1]
[0, 2]
[0, 2]
[1]
[1]
[0, 1, 3]
[3, 4]
[0, 2, 6]
[0, 2, 6]
[0, 2, 6]
[0, 3, 4, 6]
[1, 6]
[1, 5]
[1, 5]
[0, 1, 2, 6]
[1, 5]
[3, 4, 6]
[1, 4, 5]
[4, 5]
[3, 4]
[4, 5]
[1, 6]
[4, 5]
[4, 5]
[4, 5]
[1, 3]
[0, 2]
[4, 5]
[4, 5]
[0, 5]
[0, 2]
[4, 5]
[4, 5]
[1, 5]
[0, 2]
[4, 5]
[1, 3]
[1, 6]
[0, 2]
[1, 3, 4]
[1, 3, 6]
[0, 2]
[1, 4, 5]
[0, 1]
[1, 5]
[1, 3]
[1]
[1, 6]
[2, 6]
[3, 4]
[0, 1, 2, 6]
[0, 2]
[1, 6]
[3, 4, 6]
[0, 2]
[0, 2, 3, 6]
[0, 2]
[1]
[3, 4]
[3, 4]
[1, 5]
[1, 5]
[0]
[0, 2]
[1, 4, 5]
[4, 5]
[1, 4, 5]
[1]
[4, 5]
[0, 2, 6]
[1, 5]
[3, 4]
[3, 4, 6]
[1, 3]
[1, 3]
[0, 5]
[0, 2, 6]
[4, 5]
[0, 1, 5]
[1, 6]
[0

[1, 2, 6]
[1, 2, 6]
[3, 4]
[0, 2, 6]
[4, 5]
[1, 3]
[0, 1]
[1, 3]
[1, 3]
[1, 3]
[4, 5]
[0, 2, 6]
[0, 2, 6]
[0, 1, 3]
[3, 4, 6]
[1, 3]
[1, 3, 4]
[1]
[1, 3]
[3, 4, 5]
[1]
[4, 5]
[0, 3, 6]
[0, 2]
[3, 4, 6]
[4, 5]
[1]
[3, 4]
[4, 5]
[4, 5]
[3, 4, 6]
[4, 5]
[4, 5]
[4, 5]
[1, 4, 5]
[0, 1]
[3, 4, 6]
[0, 2]
[4, 5]
[4, 5]
[1, 4, 5]
[1, 5]
[1, 3]
[3, 4, 5]
[3, 4]
[3, 4, 6]
[1, 4, 5]
[1, 4, 5]
[4, 5]
[0, 2, 6]
[3, 4]
[0, 2]
[3, 4]
[1, 3]
[0, 2]
[3, 4, 5]
[3, 4]
[3, 4, 5]
[1, 3, 4]
[4, 5]
[3, 4, 6]
[3, 4]
[0, 2]
[1, 4, 5]
[3, 4]
[4, 5]
[3, 4]
[3, 4]
[4, 5]
[1, 3, 5]
[1, 3]
[4, 5]
[1, 3, 6]
[1, 4, 5]
[0, 1, 2]
[3, 4, 6]
[1, 3]
[0, 2]
[0, 2, 6]
[3, 4]
[4, 5]
[1, 3, 6]
[4, 5]
[1, 5]
[0, 1]
[4, 5]
[4, 5]
[0, 2]
[1, 3]
[1, 3]
[0, 1, 2, 6]
[1, 4, 5]
[4, 5]
[1]
[1, 4, 5]
[1, 4, 5]
[0, 2, 6]
[0, 2]
[1, 3]
[3, 4]
[4, 5]
[0, 2]
[4, 5]
[4, 5]
[0, 4, 5]
[3, 4, 6]
[3, 4, 6]
[4, 5]
[4, 5]
[4, 5]
[1, 6]
[4, 5]
[4, 5]
[1, 3]
[1, 3]
[0, 1, 5]
[1]
[3, 4, 6]
[1, 3, 6]
[4, 5]
[1, 3]
[3, 4]
[1]
[4, 5]
[1, 3, 6]
[4, 5]
[

In [104]:
print(final_topic_assign[3])
print(final_topic_assign[463])

[1, 3, 5]
[1, 3, 5]


In [93]:
def _cal_modarity(r_i):
    cul_Q = 0
    rows,cols = _network.shape
    for c_j in range(cols):
        cul_Q += (_network[r_i, c_j] - (_degrees[r_i]*_degrees[c_j] / _total_link_num)) * len(set(_final_topic_assign[r_i]).intersection(set(_final_topic_assign[c_j])))
    return cul_Q

from time import time
global _network
global _final_topic_assign
global _degrees
global _total_link_num

_total_link_num = total_link_num
_degrees = degrees
_network = network
_final_topic_assign = final_topic_assign

begin_time = time()
rows,cols = network.shape
pool = Pool(cpu_count())
cul_Q = pool.map(_cal_modarity, range(rows))
pool.close()
pool.join()

print("walks_to_pairs_with_ns_pool [%.1f s]" % (time() - begin_time))
aver_cul_Q = np.sum(cul_Q) / total_link_num
print(aver_cul_Q)

walks_to_pairs_with_ns_pool [68.3 s]
1.1754358662531461


In [ ]:
# thres_value >= 0.15
1.1754358662531461

In [170]:
num_k = W.shape[1]
final_topic_assign = {}
for i in range(node_num):
    final_topic_assign[i] = None
    tmp = []
    for j in range(num_k):
        thres_value = norw_W[i,j]
        if thres_value >= (1/7):
            tmp.append(j)
    print(tmp)
    final_topic_assign[i] = tmp

[5]
[1]
[1, 3]
[1]
[4, 5]
[1, 2, 3]
[1, 3]
[5]
[5]
[1]
[1, 5]
[0, 6]
[1, 5]
[0, 1, 2, 6]
[4, 5]
[0, 5, 6]
[1, 3]
[5]
[4, 5]
[1, 3, 4, 5]
[1]
[1, 3]
[0, 1, 2]
[4, 5]
[1, 3]
[1, 6]
[4, 5]
[1, 4, 5]
[3, 4, 5]
[1]
[0, 2, 6]
[1, 5]
[2, 6]
[5]
[1, 3]
[1]
[4, 5]
[0, 6]
[0, 6]
[1]
[1]
[5]
[0, 2]
[4, 5]
[1, 2, 6]
[1]
[5]
[4, 5]
[4, 5]
[1]
[0, 1]
[4, 5]
[1]
[0, 2, 6]
[0, 1]
[0, 2]
[1, 2, 3]
[1, 3]
[5, 6]
[4, 5]
[1, 3]
[0, 1, 2, 3, 4, 5, 6]
[1]
[1]
[0, 1, 2]
[0, 6]
[1]
[1, 5]
[1]
[1, 3]
[2, 5, 6]
[1]
[4, 5]
[4, 5]
[1, 5]
[1]
[1, 5]
[0, 2]
[1, 4, 5]
[1, 5]
[2, 5, 6]
[0, 1, 2, 3, 4, 5, 6]
[1, 3]
[1, 3]
[1, 5]
[1, 3]
[0, 1, 2, 3, 4, 5, 6]
[1, 5]
[1, 5]
[1, 2]
[1, 5]
[4, 5]
[1, 3]
[4, 5]
[1, 5]
[5]
[1]
[5]
[1]
[4, 5]
[0, 6]
[4, 5]
[1, 5, 6]
[1, 3]
[4, 5]
[1, 5]
[4, 5]
[1, 5]
[1, 5]
[1, 2, 3]
[4, 5]
[5]
[0, 4, 5]
[4, 5]
[1, 2, 3]
[1, 5]
[1, 2, 6]
[1, 5]
[1, 3]
[1, 2, 6]
[0, 2, 6]
[4, 5]
[5]
[4, 5]
[4, 5]
[4, 5]
[4, 5]
[4, 5]
[1, 3]
[0, 2]
[1, 6]
[4, 5]
[0, 6]
[3, 4, 5]
[1]
[1]
[1]
[5]
[5]
[0, 6]
[1, 6

[2, 6]
[1, 2, 5, 6]
[0, 6]
[4, 5]
[1]
[5]
[1, 5]
[1, 5]
[1, 2, 6]
[5]
[0, 1]
[1]
[1, 5]
[4, 5]
[0, 1, 2, 3, 4, 5, 6]
[1, 5]
[1, 3]
[0, 2]
[1, 3]
[0, 6]
[1]
[1]
[1]
[1]
[4, 5]
[1]
[0, 1, 2]
[1, 6]
[5]
[4, 5]
[1, 4, 5]
[1, 2, 3]
[1, 5, 6]
[0, 2, 6]
[1]
[0, 2]
[0, 1, 2, 3, 4, 5, 6]
[1]
[1]
[5]
[0, 6]
[5]
[2, 4, 5, 6]
[1]
[0, 2, 6]
[0, 2]
[1]
[1]
[2, 5]
[4, 5]
[4, 5]
[0, 2, 6]
[0, 6]
[1, 3]
[0, 6]
[0, 5, 6]
[1, 5]
[1, 3]
[1, 5]
[0, 1]
[1, 2, 6]
[0, 6]
[1, 2, 5, 6]
[0, 6]
[1, 6]
[1, 5]
[2, 4, 5]
[1, 2, 6]
[1, 5]
[1, 5]
[5]
[3, 4, 5]
[4, 5]
[0, 2]
[0, 1, 6]
[0, 1, 2]
[0, 2]
[3, 4, 5]
[1, 5, 6]
[4, 5]
[4, 5]
[0, 1, 2, 3, 4, 5, 6]
[0, 1, 2, 3, 4, 5, 6]
[1, 2, 6]
[0, 1, 2, 3, 4, 5, 6]
[0, 1, 2, 3, 4, 5, 6]
[4, 5]
[0, 1, 2, 3, 4, 5, 6]
[1]
[5]
[1, 4, 5]
[1, 5, 6]
[0, 2, 6]
[0, 2]
[0, 2, 6]
[0, 1, 2, 3, 4, 5, 6]
[0, 6]
[1, 3, 5]
[5]
[2, 6]
[1, 2, 6]
[0, 2, 6]
[4, 5]
[0, 1, 2, 3, 4, 5, 6]
[4, 5]
[1]
[1]
[1, 3]
[0, 2]
[1, 3]
[1, 5]
[0, 6]
[0, 6]
[0, 2]
[0, 2]
[0, 2]
[5]
[1]
[1, 3]
[0, 1, 2, 3, 4, 5

[4, 5]
[1, 5]
[1, 5]
[1, 3]
[1]
[1, 6]
[1, 3, 5]
[0, 6]
[1, 3]
[1]
[1, 5]
[5]
[1, 5]
[0, 1, 2, 3, 4, 5, 6]
[0, 1, 2, 3, 4, 5, 6]
[2, 4, 5]
[1]
[4, 5]
[1]
[1, 3]
[1, 3]
[1, 2, 5, 6]
[0, 2]
[5]
[0, 6]
[1, 2]
[2, 4, 5]
[0, 2]
[0, 6]
[0, 2]
[1, 5]
[2, 4, 5]
[0, 2, 5]
[1, 6]
[1, 5]
[0, 2]
[5]
[4, 5]
[0, 1, 2, 3, 4, 5, 6]
[1, 5]
[2, 5, 6]
[1]
[0, 1, 2, 3, 4, 5, 6]
[1]
[1, 5, 6]
[1, 6]
[1]
[0, 1, 2, 3, 4, 5, 6]
[0, 6]
[5]
[1, 3]
[1]
[1]
[1]
[1]
[5]
[0, 2]
[1, 2, 6]
[1, 3]
[0, 1, 2, 3, 4, 5, 6]
[1, 3]
[1]
[1]
[1, 3]
[3, 4, 5]
[1, 2, 6]
[5]
[0, 1]
[0, 6]
[0, 1, 2, 3, 4, 5, 6]
[4, 5]
[1, 5]
[0, 1, 2, 3, 4, 5, 6]
[2, 4, 5]
[4, 5]
[0, 1, 2, 3, 4, 5, 6]
[4, 5]
[1, 5]
[5]
[3, 4, 5]
[0, 1, 6]
[0, 1, 2, 3, 4, 5, 6]
[0, 6]
[3, 4, 5]
[4, 5]
[1, 4, 5]
[1, 6]
[1]
[5]
[1]
[0, 1, 2, 3, 4, 5, 6]
[1, 5]
[1, 5]
[5]
[0, 2]
[0, 1, 2, 3, 4, 5, 6]
[0, 2]
[0, 1, 2, 3, 4, 5, 6]
[1, 3]
[0, 2]
[5]
[0, 1, 2, 3, 4, 5, 6]
[1, 5]
[1, 3, 5]
[5]
[0, 1, 2, 3, 4, 5, 6]
[0, 1, 2, 3, 4, 5, 6]
[0, 6]
[1, 5]
[1]
[5]
[0, 1, 2, 3, 

In [168]:
topic_vectors = sess.run(discriminator.topic_embed)
u_vectors = sess.run(discriminator.u_node_embed)

import math
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def softmax(x):
    return math.exp(x)

num_k = W.shape[1]
final_topic_assign = {}
for i in range(node_num):
    final_topic_assign[i] = None
    tmp = []
    norm_tmp = []
    for j in range(num_k):
        #thres_value = sigmoid(np.matmul(u_vectors[i],topic_vectors[j].T))
        thres_value = softmax(np.matmul(u_vectors[i],topic_vectors[j].T))
        #print(thres_value)
        #if thres_value >= 0.1:
            #tmp.append(j)
        tmp.append(thres_value)
    tmp_sum = sum(tmp)
    tmp = [i/tmp_sum for i in tmp]
    for tmp_i in range(num_k):
        if tmp[tmp_i] >= (1/7):
            norm_tmp.append(tmp_i)
    print(norm_tmp)
    final_topic_assign[i] = norm_tmp


[3, 4, 5]
[1, 3, 6]
[1, 3]
[1, 3, 5]
[4, 5]
[0, 1, 3]
[1, 3]
[1, 3, 4, 5]
[4, 5]
[1, 2, 3, 4, 6]
[4, 5]
[0, 2]
[4, 5]
[0, 1, 2, 6]
[1, 4, 5]
[0, 1, 5]
[1, 3]
[4, 5]
[4, 5]
[1, 4, 5]
[1, 2, 3, 4, 6]
[0, 1]
[0, 2]
[4, 5]
[1, 3, 4]
[1, 6]
[4, 5]
[4, 5]
[1, 5]
[0, 1, 2, 6]
[0, 2, 3, 4, 6]
[1, 4, 5]
[0, 2, 3, 4, 6]
[4, 5]
[1, 3]
[1, 3]
[4, 5]
[0, 2, 6]
[0, 2, 6]
[1, 3]
[1, 2, 3, 4, 6]
[3, 4, 5]
[0, 2]
[4, 5]
[1]
[1, 3]
[1, 4, 5]
[1, 4, 5]
[3, 4, 5]
[0, 3, 4, 6]
[0, 2, 3, 4, 6]
[4, 5]
[1]
[0, 2, 6]
[0, 1]
[0, 2, 6]
[1]
[1, 3]
[4, 5]
[3, 4, 5]
[1, 3]
[0, 2, 3, 4, 6]
[1, 3, 4]
[1, 3, 5]
[0, 2, 3, 6]
[0, 2, 6]
[1, 3]
[3, 4, 6]
[1, 3, 4, 5]
[1, 3]
[1, 2, 4, 5, 6]
[1, 3, 5]
[1, 4, 5]
[1, 4, 5]
[1, 5]
[1, 3, 4, 6]
[1, 4, 5]
[0, 2, 6]
[1, 4, 5]
[1, 5]
[0, 1, 5]
[2, 3, 4, 6]
[1, 3]
[0, 1, 5]
[4, 5]
[0, 1, 3]
[2, 3, 4, 6]
[1, 4, 5]
[1, 5]
[1]
[1, 4, 5]
[1, 4, 5]
[1, 3]
[3, 4, 5]
[3, 4, 5]
[4, 5]
[1]
[3, 4, 5]
[0, 1, 3]
[3, 4, 5]
[0, 2, 6]
[1, 4, 5]
[1, 5]
[0, 1, 3]
[4, 5]
[3, 4]
[1, 5]
[1, 3, 4, 5]
[

[4, 5]
[2, 3, 4, 6]
[3, 4, 5]
[1, 6]
[4, 5]
[3, 4, 5]
[4, 5]
[1, 3]
[0, 2]
[4, 5]
[4, 5]
[0, 5]
[0, 2]
[4, 5]
[4, 5]
[1, 5]
[0, 2, 6]
[4, 5]
[1, 3]
[1, 3, 6]
[0, 2, 6]
[1, 3, 4, 6]
[1, 3, 6]
[0, 2, 6]
[1, 3, 4, 5]
[0, 1, 2, 6]
[1, 5]
[1, 3, 4]
[1, 3]
[1, 6]
[2, 4, 6]
[2, 3, 4, 6]
[0, 1, 2, 6]
[0, 2]
[1, 6]
[2, 3, 4, 6]
[0, 2]
[0, 2, 3, 6]
[0, 2, 6]
[1]
[3, 4, 5, 6]
[3, 4, 6]
[1, 5]
[1, 5]
[0, 2, 5]
[0, 2, 6]
[1, 4, 5]
[1, 4, 5]
[1, 4, 5]
[1, 5]
[4, 5]
[0, 2, 6]
[1, 5]
[3, 4]
[0, 2, 3, 4, 6]
[1, 3]
[1, 3]
[0, 2, 5]
[0, 2, 6]
[4, 5]
[0, 1, 5]
[1, 2, 3, 6]
[0, 2, 6]
[0, 1, 3, 6]
[0, 2, 6]
[1, 4, 5]
[1, 2, 5, 6]
[2, 3, 4, 6]
[1, 3]
[4, 5]
[0, 1, 2]
[1, 3]
[2, 3, 4, 6]
[0, 1, 3]
[4, 5]
[0, 1, 3]
[1, 3, 4, 6]
[1, 5]
[0, 2, 3, 4, 6]
[4, 5]
[3, 4, 5]
[1, 6]
[4, 5]
[3, 4, 5]
[0, 2, 6]
[3, 4]
[1]
[1, 5, 6]
[0, 1, 2, 6]
[0, 2, 6]
[0, 2, 6]
[0, 2, 4, 6]
[0, 2, 6]
[0, 2, 6]
[1, 5, 6]
[1, 5]
[1, 6]
[3, 4, 5]
[1, 5, 6]
[1, 5]
[3, 4, 5]
[4, 5]
[1, 2, 3, 4, 6]
[0, 2, 3, 4, 6]
[0, 3, 4, 6]
[3, 4, 5, 6]


[3, 4, 6]
[3, 4, 5]
[1, 4, 5]
[4, 5]
[1, 4, 5]
[1, 3, 4, 6]
[2, 3, 4, 6]
[0, 2, 6]
[0, 2, 6]
[0, 2, 6]
[0, 2]
[0, 2, 6]
[1, 3, 6]
[0, 2, 6]
[1, 5]
[1, 2, 6]
[0, 2]
[1, 4, 5]
[0, 2, 6]
[1, 4, 5]
[0, 2, 6]
[0, 2, 3, 4, 6]
[2, 3, 4, 6]
[2, 3, 4, 6]
[0, 2, 6]
[1, 4, 5]
[0, 2]
[3, 4, 5]
[0, 2]
[3, 4, 5]
[0, 2, 6]
[0, 1, 2, 6]
[1, 2, 3, 6]
[0, 2, 6]
[3, 4, 5]
[0, 2, 3, 4, 6]
[0, 2]
[1, 5]
[4, 5]
[0, 2]
[0, 2]
[1, 3, 6]
[1, 5]
[1, 5]
[4, 5]
[0, 2]
[1, 5]
[2, 3, 4, 6]
[0, 2, 3, 4, 6]
[1, 3, 6]
[1, 6]
[1, 3, 4]
[3, 4, 5]
[0, 1, 2]
[0, 2, 5]
[0, 2, 6]
[3, 4]
[0, 2]
[3, 4, 6]
[1, 3]
[3, 4, 6]
[0, 2]
[1, 3]
[3, 4]
[1, 3]
[1, 4, 5]
[1, 4, 5]
[0, 2, 3, 4, 6]
[0, 2, 6]
[1, 3]
[4, 5]
[1, 3]
[3, 4, 5]
[3, 4, 5]
[1]
[0, 2, 6]
[0]
[3, 4, 5]
[0, 1, 2, 6]
[2, 3, 4, 6]
[3, 4, 5]
[4, 5]
[1, 3]
[0, 1]
[1, 4, 5]
[0, 1, 3]
[1, 4, 5]
[1, 3]
[1]
[0, 2, 6]
[3, 4, 5]
[0, 2, 6]
[1, 3]
[0, 2]
[1, 4, 5]
[4, 5]
[4, 5]
[1, 5, 6]
[1, 3, 6]
[1, 4, 5]
[1, 3]
[1, 6]
[1, 5, 6]
[4, 5]
[2, 3, 4, 6]
[0, 1, 5]
[0, 2, 6]
[3, 4, 5

In [171]:
def _cal_modarity(r_i):
    cul_Q = 0
    rows,cols = _network.shape
    for c_j in range(cols):
        cul_Q += (_network[r_i, c_j] - (_degrees[r_i]*_degrees[c_j] / _total_link_num)) * len(set(_final_topic_assign[r_i]).intersection(set(_final_topic_assign[c_j])))
    return cul_Q

from time import time
global _network
global _final_topic_assign
global _degrees
global _total_link_num

_total_link_num = total_link_num
_degrees = degrees
_network = network
_final_topic_assign = final_topic_assign

begin_time = time()
rows,cols = network.shape
pool = Pool(cpu_count())
cul_Q = pool.map(_cal_modarity, range(rows))
pool.close()
pool.join()

print("walks_to_pairs_with_ns_pool [%.1f s]" % (time() - begin_time))
aver_cul_Q = np.sum(cul_Q) / total_link_num
print(aver_cul_Q)

walks_to_pairs_with_ns_pool [142.8 s]
0.7585394003234454


In [ ]:
# norw_W (1/7)
0.7585394003234454

# thres_value >= 0.005
0.8605865973868759

# thres_value >= 0.01
0.9275414645591384

# thres_value >= 0.05
0.937267556305917

# thres_value >= 0.1
0.7986236113326738

# thres_value >= 0.15
0.6332584684135011

# thres_value >= 0.2
0.577382059212949

# thres_value >= (1/7)
1.333728559108629

In [167]:
1/7

0.14285714285714285

In [15]:
topic_vectors = sess.run(discriminator.topic_embed)
u_vectors = sess.run(discriminator.u_node_embed)

import math
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

num_k = W.shape[1]
final_topic_assign = {}
for i in range(node_num):
    final_topic_assign[i] = None
    tmp = []
    for j in range(num_k):
        tmp.append(sigmoid(np.matmul(u_vectors[i],topic_vectors[j].T)))
    #print(tmp)
    final_topic_assign[i] = tmp.index(max(tmp))
    
print(len(set(final_topic_assign.values())))

7


In [16]:
final_topic_assign

{0: 4,
 1: 1,
 2: 1,
 3: 1,
 4: 5,
 5: 0,
 6: 1,
 7: 5,
 8: 4,
 9: 1,
 10: 5,
 11: 0,
 12: 5,
 13: 0,
 14: 5,
 15: 1,
 16: 1,
 17: 5,
 18: 5,
 19: 5,
 20: 1,
 21: 1,
 22: 0,
 23: 4,
 24: 3,
 25: 1,
 26: 5,
 27: 4,
 28: 5,
 29: 1,
 30: 0,
 31: 5,
 32: 4,
 33: 5,
 34: 1,
 35: 1,
 36: 5,
 37: 0,
 38: 0,
 39: 3,
 40: 1,
 41: 4,
 42: 0,
 43: 5,
 44: 1,
 45: 1,
 46: 5,
 47: 5,
 48: 4,
 49: 3,
 50: 0,
 51: 5,
 52: 1,
 53: 0,
 54: 0,
 55: 0,
 56: 1,
 57: 1,
 58: 4,
 59: 4,
 60: 1,
 61: 4,
 62: 1,
 63: 1,
 64: 0,
 65: 0,
 66: 1,
 67: 4,
 68: 5,
 69: 1,
 70: 1,
 71: 1,
 72: 5,
 73: 5,
 74: 5,
 75: 3,
 76: 5,
 77: 0,
 78: 5,
 79: 1,
 80: 5,
 81: 4,
 82: 1,
 83: 1,
 84: 5,
 85: 1,
 86: 4,
 87: 5,
 88: 1,
 89: 1,
 90: 5,
 91: 5,
 92: 1,
 93: 4,
 94: 4,
 95: 5,
 96: 1,
 97: 4,
 98: 1,
 99: 4,
 100: 0,
 101: 5,
 102: 1,
 103: 1,
 104: 5,
 105: 4,
 106: 1,
 107: 5,
 108: 5,
 109: 1,
 110: 5,
 111: 5,
 112: 5,
 113: 5,
 114: 1,
 115: 4,
 116: 1,
 117: 5,
 118: 1,
 119: 1,
 120: 0,
 121: 5,
 122: 5,
 12

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import numpy as np
import numpy 

class Classifier(object):

    def __init__(self, vectors, clf):
        self.embeddings = vectors
        self.clf = TopKRanker(clf)
        self.binarizer = MultiLabelBinarizer(sparse_output=True)

    def train(self, X, Y, Y_all):
        self.binarizer.fit(Y_all)
        X_train = [self.embeddings[int(x)] for x in X]
        Y = self.binarizer.transform(Y)
        self.clf.fit(X_train, Y)

    def evaluate(self, X, Y):
        top_k_list = [len(l) for l in Y]
        Y_ = self.predict(X, top_k_list)
        Y = self.binarizer.transform(Y)
        averages = ["micro", "macro", "samples", "weighted"]
        results = {}
        for average in averages:
            results[average] = f1_score(Y, Y_, average=average)
        results['accuracy'] = accuracy_score(Y, Y_)
        
        # print('Results, using embeddings of dimensionality', len(self.embeddings[X[0]]))
        # print('-------------------')
        #print(results)
        return results
        # print('-------------------')

    def predict(self, X, top_k_list):
        X_ = numpy.asarray([self.embeddings[int(x)] for x in X])
        Y = self.clf.predict(X_, top_k_list=top_k_list)
        return Y

    def split_train_evaluate(self, X, Y, train_precent, seed=0):
        state = numpy.random.get_state()

        training_size = int(train_precent * len(X))
        numpy.random.seed(seed)
        shuffle_indices = numpy.random.permutation(numpy.arange(len(X)))
        X_train = [X[shuffle_indices[i]] for i in range(training_size)]
        Y_train = [Y[shuffle_indices[i]] for i in range(training_size)]
        X_test = [X[shuffle_indices[i]] for i in range(training_size, len(X))]
        Y_test = [Y[shuffle_indices[i]] for i in range(training_size, len(X))]

        self.train(X_train, Y_train, Y)
        numpy.random.set_state(state)
        return self.evaluate(X_test, Y_test)
    
#label_file = 'group.txt'    
#label_file = '/home/tmpuser/junyang/GraphGan/GraphGAN/data/cora/cane_group_clean.txt'
#label_file = '/home/tmpuser/junyang/GraphGan/GraphGAN/data/wiki/group.txt'
#label_file = '/home/profgong/junyang/GraphGAN/data/dblp/group_clean.txt'
label_file = './../../RNE_PAKDD2019/ARNE/input/cora_group.txt'
#label_file = './group.txt'

#label_file = 'group_clean.txt'
def read_node_label(filename):
    fin = open(filename, 'r')
    X = []
    Y = []
    while 1:
        l = fin.readline()
        if l == '':
            break
        vec = l.strip().split(' ')
        X.append(vec[0])
        Y.append(vec[1:])
    fin.close()
    return X, Y
#vectors = model.vectors
#vectors = embedding
#vectors = w2v.vectors
#vectors = embeddings_concat_norm
X, Y = read_node_label(label_file)

from sklearn.multiclass import OneVsRestClassifier
class TopKRanker(OneVsRestClassifier):
    def predict(self, X, top_k_list):
        probs = numpy.asarray(super(TopKRanker, self).predict_proba(X))
        all_labels = []
        for i, k in enumerate(top_k_list):
            probs_ = probs[i, :]
            labels = self.classes_[probs_.argsort()[-k:]].tolist()
            probs_[:] = 0
            probs_[labels] = 1
            all_labels.append(probs_)
        return numpy.asarray(all_labels)


import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")